In [12]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from datetime import datetime

# Função para transformar o dataset em um problema binário (classe 0 contra as outras)
def transformar_problema_binario(y, classe_0):
    # Retorna uma série pandas para garantir que podemos usar o método .loc
    return pd.Series([1 if label == classe_0 else 0 for label in y], index=y.index)

# Função principal para análise de dados temporais
def analisar_dados_temporais(X, y, class_names, date_col, periods, classe_0=0):
    global TUDO
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(X.shape[1])])
    
    # Transforma o problema em binário e mantém a estrutura pandas
    y_binario = transformar_problema_binario(y, classe_0)

    # Adiciona a coluna de períodos aos dados
    X[date_col] = pd.to_datetime(X[date_col])
    X['periodo'] = X[date_col].dt.to_period(periods)

    TUDO = []

    # Itera por cada período
    for periodo, grupo in X.groupby('periodo'):
        print(f"\nAnalisando Período: {periodo}")
        
        # Extrai os dados do período
        X_periodo = grupo.drop(columns=['periodo', date_col])
        y_periodo = y_binario.loc[grupo.index]
        
        # Divide o dataset em treino e teste
        X_train, X_test, y_train, y_test = train_test_split(X_periodo, y_periodo, test_size=0.2, random_state=42)
        
        # Treina o modelo
        modelo = LogisticRegression(max_iter=200)
        modelo.fit(X_train, y_train)

        # Obtém as previsões
        y_pred = modelo.predict(X_test)

        # Analisa a influência das features nas decisões do modelo
        for idx in range(len(X_test)):
            instancia_test = X_test.iloc[[idx]]
            classe_predita = y_pred[idx]
            Vs = instancia_test.iloc[0].to_dict()

            # Calcula `gamma_A` usando `decision_function`
            gamma_A = modelo.decision_function(instancia_test)[0]

            # Cálculo do valor delta para cada feature
            delta = []
            w = modelo.coef_[0]
            for i, feature in enumerate(X_test.columns):
                if w[i] < 0:
                    delta.append((Vs[feature] - X_test[feature].max()) * w[i])
                else:
                    delta.append((Vs[feature] - X_test[feature].min()) * w[i])

            # Calcula R
            R = sum(delta) - gamma_A

            # Computa a PI-explicação para a instância atual
            Xpl = one_explanation(Vs, delta, R, X_test.columns.tolist(), modelo, instancia_test, X_periodo)
            
            TUDO.append(Xpl)
            
            if classe_predita != y_test.iloc[idx]:
                print(f"Instância {idx}: Mudança de Classe! Classe Verdadeira: {y_test.iloc[idx]} | Classe Predita: {classe_predita}")
            else:
                print(f"Instância {idx}: Nenhuma Mudança de Classe.")

# Função para calcular a PI-explicação
def one_explanation(Vs, delta, R, feature_names, modelo, instancia_test, X):
    Xpl = []
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)
    R_atual = R
    Idx = 0
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]
        feature = feature_names[sorted_idx]
        feature_value = Vs[feature]

        Xpl.append(f"{feature} - {feature_value} ")
        R_atual -= delta_value
        Idx += 1
    
    return Xpl

# Exemplo de uso com dados fictícios (substitua X, y, e date_col pelos dados reais)
# Criando um DataFrame de exemplo
data = {
    'feature_0': [5.1, 4.9, 4.7, 4.6, 5.0, 5.4, 5.1, 5.2],
    'feature_1': [3.5, 3.0, 3.2, 3.1, 3.6, 3.9, 3.7, 3.4],
    'feature_2': [1.4, 1.4, 1.3, 1.5, 1.4, 1.7, 1.5, 1.4],
    'feature_3': [0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.3, 0.2],
    'date': ['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04'],
    'target': [0, 0, 0, 0, 1, 1, 1, 1]
}

df = pd.DataFrame(data)
X = df.drop(columns=['target'])
y = df['target']
date_col = 'date'

# Rodando a análise por períodos
analisar_dados_temporais(X, y, class_names=[0, 1], date_col=date_col, periods='M', classe_0=0)



Analisando Período: 2020-01


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1